In [4]:
# created on Dec 24, 2020
# modified on April 14, 2021
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search existing tweets
# @reference: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

!python -m pip install tweepy


In [5]:
import tweepy, json, time, csv


In [6]:

# Create a csv file to store the structured data after processing.
csvfile = open("assets/searched_tweets.csv", "w", newline='', encoding="utf-8") # mode a, r, w

# All the fields of each data entry that I want to collect.
fieldnames = ['username', 'userid', 'profile_location', 'created_at', 'text', 'retweet_count', 'source', 'coordinates']

# Create a writer to write the structured data to the csv file.
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
# Write the header to the csv file
writer.writeheader()





In [7]:
# Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
consumer_key = "dCqLqGdBmCHlvCdNVuaoImv54"
consumer_secret = "kKPA9MtNn6F2NY4CkjNFXlh8q5lerJHFUQkGeV2C9RxbkwIe9c"
access_token = "14324013-imf2bK7r4VBk3YWxBt3QvbzwNXKF26LT12hdGUv0D"
access_token_secret = "nDW56CN7sIdGR1pTLvpreNmMmffTBctrAYSfKT3g6wwn4"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Define the search term and the date_since date as variables
search_words = "#BLM"
location = "47.6138893,-122.3107869,100mi"
# read the Twitter API document to look for other ways to customize your queries.
# refer to https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators
# for example: you can ignore all the retweets by #wildfires -filter:retweets
# Geolocalization: the search operator “near” isn’t available in the API, but there is a more precise way to restrict
# your query by a given location using the geocode parameter specified with the template “latitude,longitude,radius”,
# for example, “47.6138893,-122.3107869,10mi” (capitol hill at Seattle). When conducting geo searches, the search API will first attempt to find Tweets、
# which have lat/long within the queried geocode, and in case of not having success, it will attempt to find Tweets created
# by users whose profile location can be reverse geocoded into a lat/long within the queried geocode, meaning that is possible
# to receive Tweets which do not include lat/long information.

date_since = "2018-11-16"


# Collect tweets
# tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since).items(100)
tweets = tweepy.Cursor(api.search, q=search_words, geocode=location, lang="en", since=date_since).items(100)

# Iterate and print tweets
for tweet in tweets:
    row = {
        'username': tweet.author.name,
        'userid': tweet.author.id,
        'profile_location': tweet.author.location,
        'created_at': str(tweet.author.created_at),
        'text': tweet.text,
        'retweet_count': tweet.retweet_count,
        'source': tweet.source,
        'coordinates': tweet.coordinates
    }
    writer.writerow(row)
    print(row)


csvfile.close()
# notify the completion of the program in the console.
print("finished")

{'username': 'JenKComedy 🖤', 'userid': 1683489637, 'profile_location': 'Seattle, Wa', 'created_at': '2013-08-19 14:56:21', 'text': "HABEAS CORPUS.\n\nI don't care if he resisted arrest. In no court does that offense constitute the death penalty. THE… https://t.co/W4wKqUPUNR", 'retweet_count': 0, 'source': 'Twitter for Android', 'coordinates': None}
{'username': 'James Johnson', 'userid': 1128332041949286400, 'profile_location': 'Seattle', 'created_at': '2019-05-14 16:11:36', 'text': "@KatiePhang Makes no difference. Chauvin continued to hold his knee on Floyd's neck a full 3 minutes after he loss… https://t.co/wfvdng2BQ8", 'retweet_count': 0, 'source': 'Twitter Web App', 'coordinates': None}
{'username': 'Mona Austin', 'userid': 20745988, 'profile_location': 'National', 'created_at': '2009-02-13 03:55:51', 'text': 'BREAKING: Officer Kim Potter has been charged with 2nd degree manslaughter in the death of Daunte Wright. #BLM #DaunteWright #SayHisName', 'retweet_count': 0, 'source': 'Twi

{'username': 'James Johnson', 'userid': 1128332041949286400, 'profile_location': 'Seattle', 'created_at': '2019-05-14 16:11:36', 'text': 'RT @jimmyhawk9: "To be a Jew means always being with the oppressed and never the oppressors." ~ Marek Edelman\n\n#BLM \n#FreePalestine 🇵🇸\n#ICC…', 'retweet_count': 1, 'source': 'Twitter Web App', 'coordinates': None}
{'username': 'James Johnson', 'userid': 1128332041949286400, 'profile_location': 'Seattle', 'created_at': '2019-05-14 16:11:36', 'text': 'RT @jimmyhawk9: "Man is evil, by nature man is a beast. People have to be educated from childhood, from kindergarten, that there should be…', 'retweet_count': 4, 'source': 'Twitter Web App', 'coordinates': None}
{'username': 'James Johnson', 'userid': 1128332041949286400, 'profile_location': 'Seattle', 'created_at': '2019-05-14 16:11:36', 'text': 'RT @jimmyhawk9: The Tribulation Daily: 2017 to 2024\n\n"Only two things are infinite: the universe and human stupidity, and I\'m not sure abou…', 'retweet_co

{'username': 'James Johnson', 'userid': 1128332041949286400, 'profile_location': 'Seattle', 'created_at': '2019-05-14 16:11:36', 'text': 'Across America if you are Black and want to almost guarantee being stopped, profiled, or harassed by the Police and… https://t.co/tUd3FMYbrV', 'retweet_count': 1, 'source': 'Twitter Web App', 'coordinates': None}
{'username': 'babakofi', 'userid': 18578953, 'profile_location': 'Seattle, WA', 'created_at': '2009-01-03 13:25:46', 'text': 'Cop Fired After Pulling Gun on Army Lieutenant #BLM #PoliceBrutality #DefundThePolice\nhttps://t.co/biVMmqh2M0 https://t.co/Pmg2W7Obcf', 'retweet_count': 0, 'source': 'Hootsuite Inc.', 'coordinates': None}
{'username': 'Lisa Gant', 'userid': 1378064694087348224, 'profile_location': 'Bellingham Washington', 'created_at': '2021-04-02 19:20:06', 'text': 'I am a 40 yr. old white woman, and I am sickened by the death of yet ANOTHER young black man. I hope Chauvin goes d… https://t.co/wWkzqs0ztm', 'retweet_count': 0, 'sourc

{'username': 'Matt D.C.', 'userid': 23562316, 'profile_location': 'Seattle, WA', 'created_at': '2009-03-10 05:49:19', 'text': 'New favorite shirt. Shout out to Mahogany Mommies for the shirt. Buy this and other products here:… https://t.co/M8BJ1fX5IK', 'retweet_count': 1, 'source': 'Twitter for iPhone', 'coordinates': None}
{'username': '#HighRiskCT #CovidisAirborne #VaxUp #MaskUp', 'userid': 2740338237, 'profile_location': 'Home', 'created_at': '2014-08-12 00:04:01', 'text': 'Sorry I’ve been using the hashtag that misspelled #DaunteWright ‘s name in previous quote tweets #BLM #PoliceReformNow', 'retweet_count': 0, 'source': 'Twitter for iPhone', 'coordinates': None}
{'username': 'James Johnson', 'userid': 1128332041949286400, 'profile_location': 'Seattle', 'created_at': '2019-05-14 16:11:36', 'text': 'RT @jimmyhawk9: January 2020, when Donald Trump had been briefed that #COVID19 was a DEADLY AIRBORNE DISEASE WHICH WAS ASYMTOMATIC, He bein…', 'retweet_count': 5, 'source': 'Twitter Web 